# PostgreSQL
The goal of the data engineer is to unlock an organization's data ecosystem to a wide group of analysts, data scientists, or any other interested member. Sure, we could share the SQLite file around to every interested user, but what if there were 10,000 people interested in the data? SQLite does not scale well for that use case so we require a better choice.

To serve such a vast amount of users, we would be better off using another database engine. In this course we'll use an open source relational database management system (RDBMS) called **Postgres**. Postgres is a much more robust engine that is implemented as a server rather than a single file. As a server, Postgres accepts connections from clients who can run queries like a **SELECT, INSERT**, or any other type of SQL query making the data accessible to a wide range of people.

Using this model, Postgres can handle multiple connections to the database solving one of the main data engineering challenges.

<Img src="https://github.com/rhnyewale/Data-Engineering/blob/main/Images/postgres1.jpg?raw=true">
    
The above diagram illustrates the client-server model used by Postgres. Two users, Rose and Bruno can both be connected to the same Postgres server and access the databases it contains.

Let's explore necessary skills to interact and manage a Postgres database. This is a fundamental skill that any data engineer should have, as one of the main roles of a data engineer is ensure that data is readily available and stored in a way that makes accessing it easy and efficient. We will start by learning how to connect to a Postgres database and run simple SQL queries.
    


# Introduction
Create a table for storing data representing user accounts. The dataset that we will be using is stored in a CSV file named user_accounts.csv. Its data does not correspond to real users, it was randomly generated data using faker.

In order to communicate with our Postgres server, we will be using the open source psycopg2 Python library. You can think of **psycopg2** being similar to connecting to a SQLite database using the sqlite3 library.

To connect to the database we use the *psycopg2.connect()* function by passing it a string containing the name of the database to which we want to connect to as well as our username. So, to connect to a database named dq using psycopg2, a user named Rose we would do the following:

import psycopg2

conn = psycopg2.connect("dbname=dq user=Rose")

As you see, Rose connects to the database by specifying the database name dbname and a user user in the psycopg2.connect() function. The string "dbname=dq user=Rose" is referred to as connection string. In the above example, the connection string specifies that Rose wants to connect to a database named dq using her username, Rose.

Because Postgres supports multiple simultaneous connections, Postgres uses multiple users and databases as a way to improve security and division of data. Without those values attached, Postgres will not know who is trying to connect and where so it will fail. Once Rose is connected, she is ready to take advantage of the features Postgres has.

Once she's finished doing what she wants to do with the database, Rose should close the connection to avoid leaving useless, resource consuming connections opened. To do so, she can use the connection.close() method:

conn.close()

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2

In [3]:
# import psycopg2
# conn = psycopg2.connect("dbname=postgres user=Rose")

We opened a connection to Postgres by using the **psycopg2.connect()** function of the **psycopg2** module. This method takes a connection string as argument that the library uses to connect to the Postgres server. Its return value is a connection object.

The **connection object** allows the client to interact with the database server until it is closed.<br/>
To issue commands to the database, you will also need to create a cursor object by calling the **connection.cursor()** method.<br/> This object is the one that we will use to execute our commands.<br/> Be warned though, whenever you close the connection, you will not be able to issue anymore commands with that connection's cursor.

To execute commands on the Postgres database, you call the cursor.execute() method with a SQL query passed as a string. This string is often referred to as query string. For instance, we can select all elements from the users table like so:

`cur = conn.cursor()`<br/>
`cur.execute("SELECT * FROM users;")`

<Img src="https://github.com/rhnyewale/PostgreSQL/blob/main/Images/connection.JPG?raw=true">
    
The **cursor.execute()** method does not return the query results straight away. It will return None if the query was successful and throw an error otherwise. After executing a query, we can use the cursor object to iterate over all results using a for loop like so:

In [7]:
# cur.execute("SELECT * FROM users;")
# for row in cur:
#     print(row)

Alternatively, we can get the returned value — or values — by calling one of the two methods: 
cursor.fetchone() or cursor.fetchall().<br/> **cursor.fetchone()** method returns the first result or None if there are no results.<br/> **cursor.fetchall()** method returns a list containing each row from the result or an empty list [] if there are no rows matching the query.

In [2]:
try:
    conn = psycopg2.connect("dbname=Dataquest user=postgres password=123")
except psycopg2.Error as e:
    print("Could not make connection to Postgres Database")
    print(e)

In [3]:
cur = conn.cursor()

In [4]:
cur.execute("""
    CREATE TABLE users(
        id integer PRIMARY KEY,
        email text,
        name text,
        address text
    );
""")

If you were to inspect the Dataquest database now, you would notice that there actually isn't a table named users inside of it. This isn't a bug — it's because of a concept called SQL transactions. In short, transactions are a mechanism that Postgres uses to ensure data consistency. It makes it so that queries are grouped together and changes are only saved when explicitly requested. This contrasts with SQLite, where every query we made that modified the data was executed and immediately changed the database.

In [5]:
cur.execute('SELECT * FROM users')

ProgrammingError: relation "users" does not exist
LINE 1: SELECT * FROM users;
                      ^

## Commit()

With Postgres, we're dealing with multiple users who could be changing the database at the same time and with a remote connection to a server which could fail at any moment. In order to see why this can be a problem, imagine we're keeping track of accounts for different customers at a bank.

<Img src="https://github.com/rhnyewale/PostgreSQL/blob/main/Images/bank_eg.JPG?raw=true">
    
Then, the first query would run properly, but the second would not. The result is that Jim would be credited 100 dollars, but 100 dollars would not be removed from Sue. This would cause the bank to lose money. 
    
To avoid this problem, Postgres uses **transactions**.<br/>
Transactions prevent this type of behavior by ensuring that all the queries in a transaction block are executed together. If any query in a transaction fails, the whole transaction group fails, and no changes are made to the database at all.

After having issued some queries, to tell Postgres that we want to execute them as a transaction group we commit the changes by executing the **connection.commit()** method:
    
`conn.commit()`
    
<Img src="https://github.com/rhnyewale/PostgreSQL/blob/main/Images/transaction.gif?raw=true">
    
Notice how the values in the table do not change until the transaction is committed. Whenever we open a connection in psycopg2, a new transaction will automatically be created. All queries are grouped together up until those are committed. When a commit is executed, the Postgres engine will run all the queries at once.

You can think of committing as saving a document in a text editor. Say you open a text document and add a few lines of text. The document won't really be modified until you save the changes. This is what committing the changes in a database does.
    
If we don't want to apply the changes in the transaction block, we can call the **connection.rollback()** method to remove the transaction. Not calling either of these methods will cause the transaction to stay in a pending state, and will result in the changes not being applied to the database.

In [6]:
conn.commit()

Changes are only effective on the database if you commit them.

However, until you close the connection, your future queries using that same connection will already take these changes into account as if they had been committed. They will remain invisible for other users only until you commit. If you close the connection before committing them, then they will simply disappear. This means that during that time, not everyone will have the same view over the database.

Imagine that Rose executes the two following queries without committing and without closing the connection:

`UPDATE accounts SET balance=200 WHERE name="Jim";`<br/>
`UPDATE accounts SET balance=100 WHERE name="Sue";`

Then, if she queries the accounts table, she will see her changes. But if another user — say, Bruno — connects to the database and queries that same table, he will still see a balance of 100 for Jim and 200 for Sue. This means that both users have a different view of the database.

<Img src="https://github.com/rhnyewale/PostgreSQL/blob/main/Images/before_after_commit.JPG?raw=true">
    
 

## Postgres Workflow

<Img src="https://github.com/rhnyewale/PostgreSQL/blob/main/Images/commit.JPG?raw=true">
    
You can issue more than one query between each commit. All the queries issued between two commits are grouped together into a transaction block. A transaction block is executed when the connection.commit() method is called and its changes are applied if and only if all of the queries in it are successful. This mechanism allows users to ensure that queries that need to be executed together are executed as such.

## Inserting Values

`cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s);", (1, "hello@dataquest.io", "John", "123, Fake Street"))`

The behavior of %s is similar to that of {} in Python string formatting, they both act as placeholders for actual values.

* The second argument of cursor.execute when used should always be a sequence (e.g.: a list, a tuple) even if contains a single value:

`cur.execute("INSERT INTO foo VALUES (%s)", "bar")    # WRONG`<br/>
`cur.execute("INSERT INTO foo VALUES (%s)", ("bar"))  # WRONG`<br/>
`cur.execute("INSERT INTO foo VALUES (%s)", ("bar",)) # correct`<br/>
`cur.execute("INSERT INTO foo VALUES (%s)", ["bar"])  # correct`<br/>

* The placeholder shouldn't be quoted, Psycopg2 will take care of that:

`cur.execute("INSERT INTO numbers VALUES ('%s')", (10,)) # WRONG`<br/>
`cur.execute("INSERT INTO numbers VALUES (%s)", (10,))   # correct`<br/>

In [7]:
cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s);",(1, "hello@dataquest.io", "John", "123, Fake Street"))

In [8]:
conn.commit()

In [9]:
cur.execute("SELECT * FROM users")
cur.fetchall()

[(1, 'hello@dataquest.io', 'John', '123, Fake Street')]

## Inserting data from .csv

This dataset contains 10000 users so it would take a very long time to manually insert them one by one using the same type of query that we did in the previous screen. Fortunately, with Python thanks to the psycopg2 package and csv module there is a simple way to insert all users from a CSV file into a Postgres table.

This module allows us to easily load a CSV file into Python and then iterate over all of its rows as shown in the following code:


In [10]:
import csv 

In [11]:
with open('data/user_accounts.csv','r') as file:
    next(file) # skip csv header (first row with column titles)
    reader = csv.reader(file)
    for row in reader:
        #cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s);", (row[0], row[1], row[2], row[3])) since row is a list
        cur.execute("INSERT INTO users VALUES(%s,%s,%s,%s);",row)

UniqueViolation: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id)=(1) already exists.


In [14]:
conn.commit()

In [15]:
cur.execute("DELETE FROM users WHERE id=1;")

In [16]:
conn.commit()

In [17]:
cur.execute("SELECT * FROM users")
cur.fetchall()

[]

In [18]:
with open('data/user_accounts.csv','r') as file:
    next(file) # skip csv header (first row with column titles)
    reader = csv.reader(file)
    for row in reader:
        #cur.execute("INSERT INTO users VALUES (%s, %s, %s, %s);", (row[0], row[1], row[2], row[3])) since row is a list
        cur.execute("INSERT INTO users VALUES(%s,%s,%s,%s);",row)

In [19]:
conn.commit()

In [21]:
cur.execute("SELECT * FROM users LIMIT 5")
cur.fetchall()

[(0,
  'anna.carter@gmail.com',
  'Anna Carter',
  '27183 Craig Shore Suite 886 New Benjamin TN 92858'),
 (1,
  'joseph.kirby@yahoo.com',
  'Joseph Kirby',
  '3594 Fox Ford Apt. 192 West Kristen GA 22838-8977'),
 (2,
  'larry.cain@martinez.net',
  'Larry Cain',
  '58208 Cook Bypass West Benjaminfurt OH 25179'),
 (3,
  'johnathan.soto@yahoo.com',
  'Johnathan Soto',
  '68536 Avery Expressway Amberton PA 95197'),
 (4,
  'paula.acosta@yahoo.com',
  'Paula Acosta',
  '065 Kayla Alley Apt. 098 Walkerview NH 14274')]

In [22]:
conn.close()